<a href="https://colab.research.google.com/github/Konstantin036/WaterSurfaceNet/blob/main/projekatOG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Postavljanje okruženja i instalacija biblioteka**

Prvo, instaliraćemo sve potrebne biblioteke. Rasterio i GeoPandas su ključni za rad sa geografskim podacima, dok je albumentations odlična biblioteka za augmentaciju slika.

In [1]:
# Instalacija potrebnih biblioteka
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q rasterio geopandas matplotlib
!pip install -q albumentations

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 71.6 MB/s eta 0:00:00


 **Importovanje biblioteka i definisanje pomoćnih funkcija**

In [2]:
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import matplotlib.pyplot as plt

# Za rad sa geo-podacima
import rasterio
from rasterio.features import shapes
import geopandas as gpd
from shapely.geometry import shape

# Postavljanje uređaja (GPU ako je dostupan, inače CPU)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

 **Priprema podataka (Dataset i DataLoader)**

Ovde ćemo definisati prilagođenu klasu za učitavanje slika i maski. Očekuje se da slike i maske imaju ista imena i da se nalaze u odvojenim folderima.

In [ ]:
class WaterBodiesDataset(Dataset):
    """
    Klasa za učitavanje ortofoto snimaka i odgovarajućih maski.
    """
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir, self.images[index])
        # Pretpostavljamo da maske imaju isto ime kao slike
        mask_path = os.path.join(self.mask_dir, self.images[index])

        # Učitavanje slike i maske. OpenCV učitava u BGR formatu.
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Učitavanje maske kao grayscale.
        mask = cv2.imread(mask_path, 0)

        # Binarizacija maske: pikseli > 0 postaju 1 (voda), ostali su 0 (pozadina).
        mask[mask > 0] = 1.0

        if self.transform is not None:
            augmentations = self.transform(image=image, mask=mask)
            image = augmentations["image"]
            mask = augmentations["mask"]
            # PyTorch očekuje masku tipa LongTensor za CrossEntropyLoss
            mask = mask.long()

        return image, mask

# Definisanje augmentacija za trening i validaciju
# Augmentacije pomažu modelu da nauči invarijantnost na rotaciju, promenu osvetljenja itd.
train_transform = A.Compose(
    [
        A.Resize(512, 512),
        A.Rotate(limit=35, p=0.5),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ],
)

val_transform = A.Compose(
    [
        A.Resize(512, 512),
        A.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ],
)

Objašnjenje augmentacija:

    Normalize: Normalizacija slika koristeći srednje vrednosti i standardne devijacije ImageNet skupa. Ovo je standardna praksa kada se koristi pre-trenirani model.

    ToTensorV2: Konvertuje sliku i masku u PyTorch tenzore.

 **Učitavanje i podela podataka**

Pre pokretanja ovog dela, potrebno je da postavite vaše podatke u Colab okruženje. Možete ih uploadovati direktno ili mountovati sa Google Drive-a.

In [ ]:
# Kreiranje praznih foldera za demonstraciju
os.makedirs("/content/data/train_images", exist_ok=True)
os.makedirs("/content/data/train_masks", exist_ok=True)
os.makedirs("/content/data/val_images", exist_ok=True)
os.makedirs("/content/data/val_masks", exist_ok=True)

# !!! VAŽNO !!!
# Ovde trebate da postavite vaše slike i maske u odgovarajuće foldere.
# Za sada, kreiraćemo nekoliko lažnih slika i maski za testiranje koda.

def create_dummy_data():
    for i in range(10): # 10 slika za trening
        dummy_image = np.random.randint(0, 256, (512, 512, 3), dtype=np.uint8)
        dummy_mask = np.zeros((512, 512), dtype=np.uint8)
        # Kreiranje "vodene površine" u centru maske
        cv2.rectangle(dummy_mask, (100, 100), (400, 400), 255, -1)
        cv2.imwrite(f"/content/data/train_images/img_{i}.png", dummy_image)
        cv2.imwrite(f"/content/data/train_masks/img_{i}.png", dummy_mask)

    for i in range(3): # 3 slike za validaciju
        dummy_image = np.random.randint(0, 256, (512, 512, 3), dtype=np.uint8)
        dummy_mask = np.zeros((512, 512), dtype=np.uint8)
        cv2.rectangle(dummy_mask, (150, 150), (350, 350), 255, -1)
        cv2.imwrite(f"/content/data/val_images/img_{i}.png", dummy_image)
        cv2.imwrite(f"/content/data/val_masks/img_{i}.png", dummy_mask)

create_dummy_data()

# Putanje do vaših podataka
TRAIN_IMG_DIR = "/content/data/train_images/"
TRAIN_MASK_DIR = "/content/data/train_masks/"
VAL_IMG_DIR = "/content/data/val_images/"
VAL_MASK_DIR = "/content/data/val_masks/"

# Kreiranje Dataset i DataLoader objekata
train_dataset = WaterBodiesDataset(
    image_dir=TRAIN_IMG_DIR, mask_dir=TRAIN_MASK_DIR, transform=train_transform
)
val_dataset = WaterBodiesDataset(
    image_dir=VAL_IMG_DIR, mask_dir=VAL_MASK_DIR, transform=val_transform
)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=2)

# Prikaz preklapanja maske i ortofotoa
def show_sample(image, mask):
    # Denormalizacija slike za prikaz
    inv_normalize = A.Normalize(
        mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
        std=[1/0.229, 1/0.224, 1/0.225],
        max_pixel_value=1.0
    )
    image = inv_normalize(image=image.permute(1, 2, 0))['image']

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title("Ortofoto")
    plt.subplot(1, 2, 2)
    plt.imshow(image)
    # Prikaz maske sa providnošću
    plt.imshow(mask, cmap="jet", alpha=0.5)
    plt.title("Ortofoto sa maskom")
    plt.show()

# Prikaz jednog primera
image, mask = train_dataset[0]
show_sample(image, mask)

**Implementacija DeepLabV3 modela**

Koristimo pre-trenirani DeepLabV3 sa ResNet-50 kao osnovom. Menjamo samo klasifikator da odgovara našem broju klasa (2: pozadina i voda).

In [ ]:
def get_model(num_classes):
    # Učitavanje pre-treniranog DeepLabV3 modela
    model = models.segmentation.deeplabv3_resnet50(weights='DeepLabV3_ResNet50_Weights.DEFAULT')

    # Menjamo klasifikator da odgovara broju klasa našeg problema
    # U DeepLabV3, to je 'classifier.4' sloj
    model.classifier[4] = nn.Conv2d(256, num_classes, kernel_size=(1, 1), stride=(1, 1))

    return model

# Naš problem ima 2 klase: 0=pozadina, 1=voda
NUM_CLASSES = 2
model = get_model(num_classes=NUM_CLASSES).to(DEVICE)

**Trening modela**

Definišemo funkciju gubitka, optimizator i petlje za trening i validaciju.

In [ ]:
# Definicija hiperparametara
LEARNING_RATE = 1e-4
NUM_EPOCHS = 25 # Povećajte broj epoha za stvarne podatke
BATCH_SIZE = 4

# Funkcija gubitka i optimizator
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

def train_one_epoch(loader, model, optimizer, loss_fn, device):
    """Funkcija za jednu epohu treninga."""
    model.train()
    loop = tqdm(loader)

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=device)
        targets = targets.to(device=device)

        # Forward pass
        predictions = model(data)['out']
        loss = loss_fn(predictions, targets)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Ažuriranje progres trake
        loop.set_postfix(loss=loss.item())

# Petlja za trening
for epoch in range(NUM_EPOCHS):
    print(f"--- Epoha {epoch+1}/{NUM_EPOCHS} ---")
    train_one_epoch(train_loader, model, optimizer, loss_fn, DEVICE)
    # U pravom projektu, ovde bi se dodala i validacija nakon svake epohe.
    # Takođe, čuvanje najboljeg modela.

# Sačuvati model nakon treninga
torch.save(model.state_dict(), "deeplabv3_water_segmentation.pth")

**Evaluacija modela**

Implementiraćemo metrike IoU i F1-score.

In [ ]:
def check_accuracy(loader, model, device="cuda"):
    """
    Funkcija za računanje metrika (IoU, F1-score, tačnost) na datom skupu.
    """
    num_correct = 0
    num_pixels = 0
    dice_score = 0 # F1-score je ekvivalent Dice koeficijentu
    iou_score = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            preds = model(x)['out']
            preds = torch.argmax(preds, dim=1)

            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)

            # Računanje Dice i IoU za svaku sliku u batch-u
            for i in range(preds.shape[0]):
                pred_i = preds[i].flatten()
                y_i = y[i].flatten()

                intersection = (pred_i & y_i).sum()

                # Dice / F1-score
                dice_score += (2. * intersection) / (pred_i.sum() + y_i.sum() + 1e-8)

                # IoU / Jaccard
                union = (pred_i | y_i).sum()
                iou_score += intersection / (union + 1e-8)

    print(f"Tačnost piksela: {num_correct/num_pixels*100:.2f}%")
    print(f"Prosečan F1-score (Dice): {dice_score/len(loader.dataset):.4f}")
    print(f"Prosečan IoU (Jaccard): {iou_score/len(loader.dataset):.4f}")

    model.train()

print("\n--- Evaluacija na validacionom skupu ---")
check_accuracy(val_loader, model, device=DEVICE)

**Vizualizacija predikcija**

Ova funkcija će prikazati original, stvarnu masku i predikciju modela jednu pored druge.

In [ ]:
def save_predictions_as_imgs(loader, model, folder="saved_images/", device="cuda"):
    """
    Funkcija za čuvanje predikcija i njihovo poređenje sa originalima.
    """
    if not os.path.exists(folder):
        os.makedirs(folder)

    model.eval()
    for idx, (x, y) in enumerate(loader):
        x = x.to(device=device)
        with torch.no_grad():
            preds = model(x)['out']
            preds = torch.argmax(preds, dim=1).cpu().numpy()

        # Prikazivanje rezultata za prvu sliku u batch-u
        plt.figure(figsize=(15, 5))

        # Originalna slika (denormalizovana)
        inv_normalize = A.Normalize(
            mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
            std=[1/0.229, 1/0.224, 1/0.225]
        )
        image_to_show = inv_normalize(image=x[0].cpu().permute(1, 2, 0))['image']

        plt.subplot(1, 3, 1)
        plt.imshow(image_to_show)
        plt.title("Originalna Slika")

        # Ground Truth Maska
        plt.subplot(1, 3, 2)
        plt.imshow(y[0].squeeze(), cmap='gray')
        plt.title("Stvarna Maska")

        # Predikcija Modela
        plt.subplot(1, 3, 3)
        plt.imshow(preds[0], cmap='gray')
        plt.title("Predikcija Modela")

        plt.savefig(f"{folder}/prediction_{idx}.png")
        plt.show()

        # Prikazujemo samo nekoliko primera da ne pretrpamo izlaz
        if idx > 2:
            break

    model.train()

save_predictions_as_imgs(val_loader, model, device=DEVICE)

**Inferencija na velikim slikama (Sliding Window)**

Za slike dimenzija 10520x10520, ne možemo ih učitati cele u memoriju GPU-a. Rešenje je da se slika iseče na manje delove (npr. 512x512), izvrši predikcija na svakom delu, i zatim se rezultati spoje nazad.


In [ ]:
def predict_large_image(model, large_image_path, patch_size=512, stride=256):
    """
    Vrši predikciju na velikoj slici koristeći sliding window pristup.
    """
    model.eval()

    # Učitavanje slike koristeći OpenCV
    large_image = cv2.imread(large_image_path)
    large_image_rgb = cv2.cvtColor(large_image, cv2.COLOR_BGR2RGB)
    h, w, _ = large_image_rgb.shape

    # Kreiranje prazne mape za predikcije
    prediction_map = np.zeros((h, w), dtype=np.uint8)

    # Normalizacija za model
    transform = A.Compose([
        A.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ])

    for y in tqdm(range(0, h, stride)):
        for x in range(0, w, stride)):
            y_end = min(y + patch_size, h)
            x_end = min(x + patch_size, w)
            patch = large_image_rgb[y:y_end, x:x_end]

            # Preskoči ako je patch manji od minimalne veličine
            if patch.shape[0] < 64 or patch.shape[1] < 64:
                continue

            # Pad-ovanje patch-a ako nije tačne veličine 512x512
            padded_patch = cv2.copyMakeBorder(
                patch, 0, patch_size - patch.shape[0], 0, patch_size - patch.shape[1],
                cv2.BORDER_CONSTANT, value=0
            )

            # Priprema za model
            input_tensor = transform(image=padded_patch)["image"].unsqueeze(0).to(DEVICE)

            with torch.no_grad():
                output = model(input_tensor)['out']
                pred_mask = torch.argmax(output, dim=1).squeeze(0).cpu().numpy().astype(np.uint8)

            # Isecanje originalne veličine patch-a iz predikcije
            pred_mask_cropped = pred_mask[:patch.shape[0], :patch.shape[1]]

            # Postavljanje predikcije na odgovarajuće mesto u finalnoj mapi
            prediction_map[y:y_end, x:x_end] = pred_mask_cropped

    return prediction_map

# Primer upotrebe (potrebno je imati veliku sliku na putanji)
# large_image_path = "/content/velika_slika.tif"
# final_prediction = predict_large_image(model, large_image_path)
# plt.imshow(final_prediction, cmap='gray')
# plt.show()
# cv2.imwrite("/content/velika_slika_predikcija.png", final_prediction * 255)

**Post-procesiranje (Raster u Vektor)**

Konačno, konvertujemo finalnu rastersku predikciju u poligone (vektorski format).


In [ ]:
def raster_to_vector(raster_path, output_vector_path):
    """
    Konvertuje binarni raster u vektorski format (GeoJSON).
    """
    with rasterio.open(raster_path) as src:
        # Čitanje prvog kanala (band)
        image = src.read(1)

        # Maska gde su vrednosti 1 (voda)
        mask = image == 1

        # Generisanje oblika (poligona) iz maske
        results = (
            {'properties': {'raster_val': v}, 'geometry': s}
            for i, (s, v) in enumerate(
                shapes(image, mask=mask, transform=src.transform))
        )

        geometries = list(results)

        # Kreiranje GeoDataFrame-a
        if geometries:
            gdf = gpd.GeoDataFrame.from_features(geometries)
            gdf.set_crs(src.crs, inplace=True)
            gdf.to_file(output_vector_path, driver='GeoJSON')
            print(f"Vektori sačuvani u: {output_vector_path}")
        else:
            print("Nije pronađena nijedna vodena površina za vektorizaciju.")

# Kreiranje lažne rasterske predikcije za demonstraciju
dummy_prediction = np.zeros((512, 512), dtype=np.uint8)
cv2.rectangle(dummy_prediction, (100, 100), (300, 300), 1, -1) # Voda je vrednost 1
cv2.circle(dummy_prediction, (400, 400), 50, 1, -1)

# Za raster_to_vector je potreban geo-referenciran fajl, pa ga kreiramo
with rasterio.open(
    '/content/dummy_prediction.tif', 'w',
    driver='GTiff', height=dummy_prediction.shape[0], width=dummy_prediction.shape[1],
    count=1, dtype=str(dummy_prediction.dtype),
    crs='+proj=utm +zone=34 +ellps=WGS84 +datum=WGS84 +units=m +no_defs' # Primer CRS
) as dst:
    dst.write(dummy_prediction, 1)


# Pokretanje konverzije
raster_to_vector('/content/dummy_prediction.tif', '/content/vodene_povrsine.geojson')